In [73]:
#Get the Pacers team_id
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

# Select the dictionary for the Pacers, which contains their team ID
pacers = [team for team in nba_teams if team['abbreviation'] == 'IND'][0]
pacers_id = pacers['id']
print(f'pacers_id: {pacers_id}')

lakers = [team for team in nba_teams if team['abbreviation'] == 'LAL'][0]
lakers_id = lakers['id']
print(f'lakers_id: {lakers_id}')

pacers_id: 1610612754
lakers_id: 1610612747


In [2]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=pacers_id,
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular)  

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')

Searching through 61 game(s) for the game_id of 0021800826 where IND vs. CLE


In [12]:
# Query for the play by play of that most recent regular season game
from nba_api.stats.endpoints import playbyplay
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
df.head() #just looking at the head of the data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0021800826,2,12,0,1,7:11 PM,12:00,None,None,None,None,None
1,0021800826,4,10,0,1,7:11 PM,12:00,Jump Ball Turner vs. Nance Jr.: Tip to Young,None,None,None,None
2,0021800826,8,6,2,1,7:11 PM,11:41,None,None,Nance Jr. S.FOUL (P1.T1) (J.Van Duyne),None,None
3,0021800826,10,3,11,1,7:11 PM,11:41,Bogdanovic Free Throw 1 of 2 (1 PTS),None,None,0 - 1,1
4,0021800826,11,3,12,1,7:12 PM,11:41,Bogdanovic Free Throw 2 of 2 (2 PTS),None,None,0 - 2,2


In [5]:
#Since the datset is fairly large you'll see plenty of elipses(...). 
#If that's the case, you can set the following options to expand the data 
#You can adjust these as you'd like
import pandas
pandas.set_option('display.max_colwidth',250)
pandas.set_option('display.max_rows',250)

In [6]:
#List unique values in the df['EVENTMSGTYPE'] colum
print(f'EVENTMSGTYPE: {sorted(df.EVENTMSGTYPE.unique())}')

EVENTMSGTYPE: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13]


In [7]:
#For quick refernce, here's an Enum for `EVENTMSGTYPE`
#This list may be incomplete as a thourogh play by play scan is necessary

from enum import Enum

class EventMsgType(Enum):
    FIELD_GOAL_MADE = 1
    FIELD_GOAL_MISSED = 2
    FREE_THROWfree_throw_attempt = 3
    REBOUND = 4
    TURNOVER = 5
    FOUL = 6
    VIOLATION = 7
    SUBSTITUTION = 8
    TIMEOUT = 9
    JUMP_BALL = 10
    EJECTION = 11
    PERIOD_BEGIN = 12
    PERIOD_END = 13

In [8]:
#pull the data for a specfic EVENTMSGTYPE
df.loc[df['EVENTMSGTYPE'] == 2].head() #hint: use the EVENTMSGTYPE values above to see different data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
5,0021800826,12,2,1,1,7:12 PM,11:21,None,None,MISS Nance Jr. 17' Jump Shot,None,None
7,0021800826,14,2,1,1,7:12 PM,11:13,MISS Evans 29' 3PT Jump Shot,None,None,None,None
14,0021800826,27,2,78,1,7:14 PM,9:47,None,None,MISS Nwaba 8' Floating Jump Shot,None,None
21,0021800826,37,2,78,1,7:18 PM,8:58,MISS Young 6' Floating Jump Shot,None,None,None,None
27,0021800826,45,2,5,1,7:20 PM,7:36,None,None,MISS Nwaba 8' Layup,None,None


In [9]:
#List unique values in the df['EVENTMSGTYPE'] column
emt_df = df.loc[df['EVENTMSGTYPE'] == 1]
print(f'EVENTMSGACTIONTYPE: {sorted(emt_df.EVENTMSGACTIONTYPE.unique())}')

EVENTMSGACTIONTYPE: [1, 2, 3, 5, 6, 7, 9, 44, 50, 52, 57, 66, 71, 72, 73, 75, 76, 79, 80, 98, 101, 108]


In [13]:
#pull the data for a specfic EVENTMSGTYPE
df.loc[df['EVENTMSGTYPE'] == 8].head() #hint: use the EVENTMSGTYPE values above to see different data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
40,0021800826,61,8,0,1,7:21 PM,6:37,None,None,SUB: Zizic FOR Chriss,None,None
47,0021800826,72,8,0,1,7:23 PM,5:23,SUB: Joseph FOR Evans,None,None,None,None
48,0021800826,73,8,0,1,7:23 PM,5:23,None,None,SUB: Clarkson FOR Nwaba,None,None
61,0021800826,92,8,0,1,7:28 PM,3:28,SUB: Holiday FOR Collison,None,None,None,None
62,0021800826,93,8,0,1,7:28 PM,3:28,SUB: Sumner FOR Young,None,None,None,None


In [16]:
len(df.columns)

12

In [17]:
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2(game_id)
pbp = pbp.get_data_frames()[0]
pbp.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,0021800826,2,12,0,1,7:11 PM,12:00,None,None,None,...,None,None,None,0,0,None,NaN,None,None,None
1,0021800826,4,10,0,1,7:11 PM,12:00,Jump Ball Turner vs. Nance Jr.: Tip to Young,None,None,...,Cleveland,Cavaliers,CLE,4,201152,Thaddeus Young,1.610613e+09,Indiana,Pacers,IND
2,0021800826,8,6,2,1,7:11 PM,11:41,None,None,Nance Jr. S.FOUL (P1.T1) (J.Van Duyne),...,Indiana,Pacers,IND,1,0,None,NaN,None,None,None
3,0021800826,10,3,11,1,7:11 PM,11:41,Bogdanovic Free Throw 1 of 2 (1 PTS),None,None,...,None,None,None,0,0,None,NaN,None,None,None
4,0021800826,11,3,12,1,7:12 PM,11:41,Bogdanovic Free Throw 2 of 2 (2 PTS),None,None,...,None,None,None,0,0,None,NaN,None,None,None


In [19]:
list(pbp.columns)

['GAME_ID',
 'EVENTNUM',
 'EVENTMSGTYPE',
 'EVENTMSGACTIONTYPE',
 'PERIOD',
 'WCTIMESTRING',
 'PCTIMESTRING',
 'HOMEDESCRIPTION',
 'NEUTRALDESCRIPTION',
 'VISITORDESCRIPTION',
 'SCORE',
 'SCOREMARGIN',
 'PERSON1TYPE',
 'PLAYER1_ID',
 'PLAYER1_NAME',
 'PLAYER1_TEAM_ID',
 'PLAYER1_TEAM_CITY',
 'PLAYER1_TEAM_NICKNAME',
 'PLAYER1_TEAM_ABBREVIATION',
 'PERSON2TYPE',
 'PLAYER2_ID',
 'PLAYER2_NAME',
 'PLAYER2_TEAM_ID',
 'PLAYER2_TEAM_CITY',
 'PLAYER2_TEAM_NICKNAME',
 'PLAYER2_TEAM_ABBREVIATION',
 'PERSON3TYPE',
 'PLAYER3_ID',
 'PLAYER3_NAME',
 'PLAYER3_TEAM_ID',
 'PLAYER3_TEAM_CITY',
 'PLAYER3_TEAM_NICKNAME',
 'PLAYER3_TEAM_ABBREVIATION']

In [25]:
from nba_api.stats.endpoints import teamdashlineups

In [39]:
help(teamdashlineups)

Help on module nba_api.stats.endpoints.teamdashlineups in nba_api.stats.endpoints:

NAME
    nba_api.stats.endpoints.teamdashlineups

CLASSES
    nba_api.stats.endpoints._base.Endpoint(builtins.object)
        TeamDashLineups
    
    class TeamDashLineups(nba_api.stats.endpoints._base.Endpoint)
     |  Method resolution order:
     |      TeamDashLineups
     |      nba_api.stats.endpoints._base.Endpoint
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, team_id, group_quantity='5', last_n_games='0', measure_type_detailed_defense='Base', month='0', opponent_team_id=0, pace_adjust='N', per_mode_detailed='Totals', period='0', plus_minus='N', rank='N', season='2018-19', season_type_all_star='Regular Season', date_from_nullable='', date_to_nullable='', game_id_nullable='', game_segment_nullable='', league_id_nullable='', location_nullable='', outcome_nullable='', po_round_nullable='', season_segment_nullable='', shot_clock_range_nullable='',

In [35]:
teamdashlineups.TeamDashLineups

nba_api.stats.endpoints.teamdashlineups.TeamDashLineups

In [74]:
df2 = teamdashlineups.TeamDashLineups(lakers_id).get_data_frames()[1]
df2.head()

,GROUP_SET,GROUP_ID,GROUP_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-2544-201580-1627742-1628366-1628398-,L. James - J. McGee - B. Ingram - L. Ball - K. Kuzma,18,13,5,0.722,233.900000,216,433,...,1,1,532,1,1,532,532,1,1,332
1,Lineups,-2544-201580-1628366-1628398-1628404-,L. James - J. McGee - L. Ball - K. Kuzma - J. Hart,15,8,7,0.533,133.493333,131,258,...,2,2,531,2,2,531,531,2,2,1
2,Lineups,-201580-1627742-1628366-1628398-1628404-,J. McGee - B. Ingram - L. Ball - K. Kuzma - J. Hart,9,4,5,0.444,82.505000,71,159,...,3,3,530,3,3,522,530,6,3,532
3,Lineups,-2199-2544-1628366-1628398-1628404-,T. Chandler - L. James - L. Ball - K. Kuzma - J. Hart,13,7,6,0.538,65.525000,60,119,...,5,6,528,4,6,529,528,3,4,134
4,Lineups,-2199-2544-1627742-1628366-1628398-,T. Chandler - L. James - B. Ingram - L. Ball - K. Kuzma,15,12,3,0.800,61.270000,56,116,...,4,5,529,10,3,530,529,5,5,25


In [71]:
print(list(df2.columns))

['GROUP_SET', 'GROUP_ID', 'GROUP_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK']


In [76]:
df2.sort_values('GP', ascending = False)

,GROUP_SET,GROUP_ID,GROUP_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-2544-201580-1627742-1628366-1628398-,L. James - J. McGee - B. Ingram - L. Ball - K. Kuzma,18,13,5,0.722,233.900000,216,433,...,1,1,532,1,1,532,532,1,1,332
4,Lineups,-2199-2544-1627742-1628366-1628398-,T. Chandler - L. James - B. Ingram - L. Ball - K. Kuzma,15,12,3,0.800,61.270000,56,116,...,4,5,529,10,3,530,529,5,5,25
1,Lineups,-2544-201580-1628366-1628398-1628404-,L. James - J. McGee - L. Ball - K. Kuzma - J. Hart,15,8,7,0.533,133.493333,131,258,...,2,2,531,2,2,531,531,2,2,1
3,Lineups,-2199-2544-1628366-1628398-1628404-,T. Chandler - L. James - L. Ball - K. Kuzma - J. Hart,13,7,6,0.538,65.525000,60,119,...,5,6,528,4,6,529,528,3,4,134
8,Lineups,-2199-2544-203484-1628366-1628398-,T. Chandler - L. James - K. Caldwell-Pope - L. Ball - K. Kuzma,13,7,6,0.538,38.866667,35,70,...,12,7,513,5,17,522,514,9,9,7
15,Lineups,-2199-203484-1627742-1628398-1628404-,T. Chandler - K. Caldwell-Pope - B. Ingram - K. Kuzma - J. Hart,10,4,6,0.400,31.101667,29,57,...,14,16,521,17,46,476,527,9,10,89
40,Lineups,-2199-202362-203484-1628366-1628404-,T. Chandler - L. Stephenson - K. Caldwell-Pope - L. Ball - J. Hart,10,4,6,0.400,13.516667,10,31,...,55,70,474,46,46,1,492,68,48,531
9,Lineups,-2199-202362-203484-1627742-1628404-,T. Chandler - L. Stephenson - K. Caldwell-Pope - B. Ingram - J. Hart,9,4,5,0.444,37.628333,31,72,...,10,9,507,17,17,504,518,7,8,25
27,Lineups,-2199-203484-1628366-1628398-1628404-,T. Chandler - K. Caldwell-Pope - L. Ball - K. Kuzma - J. Hart,9,2,7,0.222,19.558333,16,38,...,23,29,507,46,137,398,492,22,28,514
11,Lineups,-2544-201580-203484-1628366-1628398-,L. James - J. McGee - K. Caldwell-Pope - L. Ball - K. Kuzma,9,6,3,0.667,34.283333,29,67,...,19,21,513,8,17,476,505,11,18,70
